In [1]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import pandas as pd


In [2]:
data = pd.read_csv('creditcard.csv')

# Feature Scaling
scaler = StandardScaler()
data['scaled_amount'] = scaler.fit_transform(data['Amount'].values.reshape(-1, 1))
data['scaled_time'] = scaler.fit_transform(data['Time'].values.reshape(-1, 1))
data = data.drop(['Amount', 'Time'], axis=1)

# Split the data into features and target
X = data.drop('Class', axis=1)
y = data['Class']


In [3]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

# Check the balance
print(f'Original dataset shape: {y.value_counts()}')
print(f'Resampled dataset shape: {y_res.value_counts()}')


Original dataset shape: Class
0    284315
1       492
Name: count, dtype: int64
Resampled dataset shape: Class
0    284315
1    284315
Name: count, dtype: int64


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=42)


In [6]:
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)

print("Training the model...")
model.fit(X_train, y_train)
print("Model training completed.")

Training the model...
Model training completed.


In [7]:
y_pred = model.predict(X_test)

print('Classification Report:')
print(classification_report(y_test, y_pred))

print('Accuracy:', accuracy_score(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     85149
           1       1.00      0.98      0.99     85440

    accuracy                           0.99    170589
   macro avg       0.99      0.99      0.99    170589
weighted avg       0.99      0.99      0.99    170589

Accuracy: 0.9892138414551935
Confusion Matrix:
[[85009   140]
 [ 1700 83740]]


In [8]:
import pickle

# Assuming `model` is your trained RandomForest model
with open('model/fraud_model.pkl', 'wb') as f:
    pickle.dump(model, f)
